<a href="https://colab.research.google.com/github/kartik-kumarr/BreedPredictor/blob/main/DogBreedPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gdown


In [ ]:
import gdown


file_id = '1y_w4F99K2M8j-Mq1Gl2tl3wxG4z7SoQ9'

gdown.download(f'https://drive.google.com/uc?export=download&id={file_id}', 'dog_dataset.zip', quiet=False)

In [ ]:
!unzip /content/dog_dataset.zip -d /content/dogs/

In [ ]:
import os
dogs_folder = '/content/dogs/'
files_in_folder = os.listdir(dogs_folder)


print(files_in_folder)

In [ ]:
train_data_dir = '/content/dogs/images/Images/'
img_width, img_height = 224, 224
channels = 3
batch_size = 64
num_images= 50
image_arr_size= img_width * img_height * channels

In [ ]:
import os
import pandas as pd
import numpy as np
from numpy import expand_dims
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
from IPython.display import SVG
from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.utils import to_categorical, model_to_dot, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau

In [ ]:
def get_images(image_dir):

    image_index = 0
    image_arr_size= img_width * img_height * channels
    images = np.ndarray(shape=(num_images, image_arr_size))
    labels = np.array([])

    for type in os.listdir(image_dir)[:50]:
        type_images = os.listdir(image_dir + type)
        labels= np.append(labels, type.split('-')[1])

        for image in type_images[:1]:
            image_file = os.path.join(image_dir, type + '/', image)
            image_data = mpimg.imread(image_file)
            image_resized = resize(image_data, (img_width, img_height), anti_aliasing=True)
            images[image_index, :] = image_resized.flatten()
            print (type, ':', image)
            image_index += 1

    return (images, labels)

def plot_images(instances, images_per_row=10, **options):
    size = img_width
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(img_width, img_height, channels) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((img_width, img_height * n_empty)))
    for row in range(n_rows):
        if (row == len(instances)/images_per_row):
            break
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.figure(figsize=(20,20))
    plt.imshow(image, **options)
    plt.axis("off")
    plt.savefig('dogs_images.png', transparent= True, bbox_inches= 'tight', dpi= 900)
    plt.show()

In [ ]:
images, labels = get_images(train_data_dir)
plot_images(images)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True,
    rotation_range= 20,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
    validation_split=0.2,

)

valid_datagen = ImageDataGenerator(
    rescale= 1./255,
    validation_split=0.2,
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size= (img_width, img_height),
    color_mode= 'rgb',
    batch_size= batch_size,
    class_mode= 'categorical',
    subset='training',
    shuffle= True,
    seed= 1337
)

valid_generator = valid_datagen.flow_from_directory(
    train_data_dir,
    target_size= (img_width, img_height),
    color_mode= 'rgb',
    batch_size= batch_size,
    class_mode= 'categorical',
    subset='validation',
    shuffle= True,
    seed= 1337
)

In [ ]:
# num_classes = len(train_generator.class_indices)
# train_labels = train_generator.classes
# train_labels = to_categorical(train_labels, num_classes=num_classes)
# valid_labels = valid_generator.classes
# valid_labels = to_categorical(valid_labels, num_classes=num_classes)
# nb_train_samples = len(train_generator.filenames)
# nb_valid_samples = len(valid_generator.filenames)

In [ ]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
# from tensorflow.keras.applications import InceptionV3
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.utils import to_categorical

# # Define the image dimensions (replace img_width and img_height with actual values)
# img_width = 224  # Example: Replace with your image width
# img_height = 224  # Example: Replace with your image height
# num_classes = 120  # Replace with your actual number of classes (dog breeds)
# batch_size = 32  # Define your batch size


# # Load InceptionV3 as the base model, without the top layers (classification head)
# base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# # Freeze the base model layers (you can fine-tune later if needed)
# base_model.trainable = False

# # Add custom layers on top of InceptionV3
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(num_classes, activation='softmax')(x)

# # Create the final model
# model = Model(inputs=base_model.input, outputs=x)

# # Compile the model
# model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# # Print model summary
# model.summary()

# # Data augmentation and image preprocessing
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     validation_split=0.2
# )

# valid_datagen = ImageDataGenerator(
#     rescale=1./255,
#     validation_split=0.2
# )

# # Training data generator
# train_generator = train_datagen.flow_from_directory(
#     train_data_dir,
#     target_size=(img_width, img_height),
#     color_mode='rgb',
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset='training',
#     shuffle=True,
#     seed=1337
# )

# # Validation data generator
# valid_generator = valid_datagen.flow_from_directory(
#     train_data_dir,
#     target_size=(img_width, img_height),
#     color_mode='rgb',
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset='validation',
#     shuffle=True,
#     seed=1337
# )

# # Train the model
# history = model.fit(
#     train_generator,
#     steps_per_epoch=len(train_generator),
#     epochs=22,
#     validation_data=valid_generator,
#     validation_steps=len(valid_generator),
# )

# # Evaluate the model on the validation data
# score = model.evaluate(valid_generator, verbose=1)
# print("Validation Loss:", score[0])
# print("Validation Accuracy:", score[1])

In [ ]:
# import matplotlib.pyplot as plt

# # Plot training & validation accuracy values
# plt.figure(figsize=(12, 4))

# plt.subplot(1, 2, 1)
# plt.plot(history.history['accuracy'], label='Training Accuracy')
# plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
# plt.title('Training and Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()

# # Plot training & validation loss values
# plt.subplot(1, 2, 2)
# plt.plot(history.history['loss'], label='Training Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Training and Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# plt.tight_layout()
# plt.show()


In [ ]:
# ## Save the model weights
# model.save_weights('/content/dogsModel.weights.h5')

In [ ]:
!pip install gdown
!gdown --id 1ktXYuNoNngaWuyEtdjh5Vu5_iUbPStbX -O /content/dogsModel.weights.h5

print("Downloaded files:", os.listdir("/content/"))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files
import time
from termcolor import colored

img_width, img_height = 224, 224
num_classes = 120

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)
model.load_weights('/content/dogsModel.weights.h5')

def predict_dog_breed(image_path, model):
    img = image.load_img(image_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    class_labels = list(train_generator.class_indices.keys())
    predicted_breed = class_labels[predicted_class]
    plt.imshow(img)
    plt.title(f"Predicted Breed: {predicted_breed}")
    plt.axis('off')
    plt.show()
    return predicted_breed
uploaded = files.upload()

if uploaded:
    image_path = list(uploaded.keys())[0]
    print(colored("\n🎉🎂 Happy Birthday to Grapes(🍇) and Ting! 🎂🎉\n", "magenta", attrs=["bold"]))
    print(colored("✨ Let's see which majestic breed your beloved dog is! ✨\n", "cyan", attrs=["bold"]))

    time.sleep(1)
    print(colored(f"📸 Uploading image: {image_path} ...", "yellow"))
    time.sleep(1)

    predicted_breed = predict_dog_breed(image_path, model)

    print(colored("\n🐶 Drumroll please... 🥁🥁🥁\n", "blue", attrs=["bold"]))
    time.sleep(2)
    print(colored(f"🎊 The predicted breed is: {predicted_breed} 🎊", "green", attrs=["bold", "underline"]))

    print(colored("\n Wishing you and Grapes the best day ever!", "red", attrs=["bold"]))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


score = model.evaluate(valid_generator, verbose=0)
print(f"Result is {score[1] * 100:.2f}% accurate.")